#### Код реализует автоматическую арбитражную торговлю между биржами Bybit и Binance для SOL

#### Расчет спреда. Формула спреда: (Цена Binance - Цена Bybit) / Цена Bybit × 100%

## Алгоритм работы по шагам:
### 1. Мониторинг цен
* Каждые 1 секунд получает текущие цены SOL с обеих бирж
* Вычисляем спред в процентах
* Логируем текущую ситуацию

### 2. Определение возможности арбитража
* Вход в позицию: когда спред ≥ 0.02% (минимальный порог)
* Выход из позиции: когда спред ≤ 0.01% (спред сузился)

### 3. Стратегии входа
Если Binance дороже Bybit (положительный спред):
* LONG на Bybit (покупка)
* SHORT на Binance (продажа)

Если Bybit дороже Binance (отрицательный спред):
* SHORT на Bybit (продажа)
* LONG на Binance (покупка)

### 4. Расчет размера позиции
* Формула: Количество SOL = $500 / Цена SOL
* Округление до 0.1 SOL для совместимости с биржами

### 5. Исполнение сделок
* Размещаем ордера одновременно на обеих биржах
* Если один ордер не прошел - отменяет второй
* Фиксирует параметры входа (спред, размер, тип позиции)

### 6. Выход из позиции
* При сужении спреда до 0.1% закрывает обе позиции
* Рассчитываем приблизительную прибыль: |Спред входа| - |Текущий спред|

### 7. Управление рисками
* Торгуем фиксированными суммами ($500 на каждой бирже)
* Использует рыночные ордера для быстрого исполнения
* Ведем подробное логирование всех операций

In [ ]:
import time
import logging
import json
from datetime import datetime
import requests
import hmac
import hashlib
from urllib.parse import urlencode
import threading
from decimal import Decimal, ROUND_DOWN

# ==================== НАСТРОЙКИ ====================

# API ключи Bybit (Perpetual)
BYBIT_API_KEY = ""
BYBIT_SECRET_KEY = ""

# API ключи Binance (Futures)
BINANCE_API_KEY = ""
BINANCE_SECRET_KEY = ""

# Торговые настройки
SYMBOL = "SOL"
BYBIT_SYMBOL = "SOLUSDT"
BINANCE_SYMBOL = "SOLUSDT"

MIN_SPREAD_PERCENT = 0.02  # Минимальный спред в % для входа в позицию
POSITION_SIZE_USD = 500  # Размер позиции в USD на каждой бирже

# Технические настройки
CHECK_INTERVAL = 1  # Интервал проверки цен в секундах
LOG_LEVEL = logging.INFO

# URLs
BYBIT_BASE_URL = "https://api.bybit.com"
BINANCE_BASE_URL = "https://fapi.binance.com"

# ==================== ЛОГИРОВАНИЕ ====================

logging.basicConfig(
    level=LOG_LEVEL,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('arbitrage.log'),
        logging.StreamHandler()
    ]
)

# Отключаем DEBUG логи для HTTP запросов
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("requests").setLevel(logging.WARNING)

logger = logging.getLogger(__name__)

# ==================== КЛАССЫ БИРЖ ====================

class BybitPerpetualAPI:
    def __init__(self, api_key, secret_key):
        self.api_key = api_key
        self.secret_key = secret_key
        self.base_url = BYBIT_BASE_URL
        
    def _make_request(self, method, endpoint, params=None):
        timestamp = str(int(time.time() * 1000))
        recv_window = "5000"
        
        headers = {
            'X-BAPI-API-KEY': self.api_key,
            'X-BAPI-TIMESTAMP': timestamp,
            'X-BAPI-RECV-WINDOW': recv_window,
        }
        
        # Для подписанных запросов
        if endpoint in ['/v5/account/wallet-balance', '/v5/order/create', '/v5/order/cancel', '/v5/position/set-leverage', '/v5/position/list']:
            if method == "POST" and params:
                headers['Content-Type'] = 'application/json'
                payload = json.dumps(params)
                sign_string = timestamp + self.api_key + recv_window + payload
            elif method == "GET" and params:
                query_string = urlencode(sorted(params.items()))
                sign_string = timestamp + self.api_key + recv_window + query_string
            else:
                sign_string = timestamp + self.api_key + recv_window
            
            signature = hmac.new(
                self.secret_key.encode('utf-8'),
                sign_string.encode('utf-8'),
                hashlib.sha256
            ).hexdigest()
            headers['X-BAPI-SIGN'] = signature
        
        url = f"{self.base_url}{endpoint}"
        
        try:
            if method == "GET":
                response = requests.get(url, params=params, headers=headers, timeout=10)
            elif method == "POST":
                response = requests.post(url, json=params, headers=headers, timeout=10)
            
            response.raise_for_status()
            result = response.json()
            return result
        except Exception as e:
            logger.error(f"Bybit API error: {e}")
            return None
    
    def get_ticker_price(self, symbol):
        """Получить текущую цену"""
        try:
            response = requests.get(f"{self.base_url}/v5/market/tickers", 
                                  params={"category": "linear", "symbol": symbol}, timeout=10)
            data = response.json()
            
            if data.get('retCode') == 0 and data.get('result') and data['result'].get('list'):
                price = data['result']['list'][0]['lastPrice']
                return float(price)
        except Exception as e:
            logger.error(f"Error getting Bybit price: {e}")
        return None
    
    def set_leverage(self, symbol, leverage):
        """Установить плечо"""
        params = {
            'category': 'linear',
            'symbol': symbol,
            'buyLeverage': str(leverage),
            'sellLeverage': str(leverage)
        }
        
        result = self._make_request("POST", "/v5/position/set-leverage", params)
        if result and result.get('retCode') == 0:
            logger.info(f"Bybit leverage set to {leverage}x for {symbol}")
            return True
        else:
            logger.error(f"Failed to set Bybit leverage: {result}")
            return False
    
    def get_balance(self):
        """Получить баланс USDT"""
        result = self._make_request("GET", "/v5/account/wallet-balance", {"accountType": "CONTRACT"})
        if result and result.get('retCode') == 0:
            try:
                coins = result['result']['list'][0]['coin']
                for coin in coins:
                    if coin['coin'] == 'USDT':
                        return float(coin['walletBalance'])
            except (KeyError, IndexError):
                logger.error("Error parsing Bybit balance response")
        return 0
    
    def get_position(self, symbol):
        """Получить информацию о позиции"""
        result = self._make_request("GET", "/v5/position/list", {"category": "linear", "symbol": symbol})
        if result and result.get('retCode') == 0:
            try:
                positions = result['result']['list']
                if positions:
                    position = positions[0]
                    # Проверяем на пустые значения и конвертируем безопасно
                    size = float(position['size']) if position['size'] else 0.0
                    avg_price = float(position['avgPrice']) if position['avgPrice'] else 0.0
                    unrealized_pnl = float(position['unrealisedPnl']) if position['unrealisedPnl'] else 0.0
                    
                    return {
                        'size': size,
                        'side': position['side'],
                        'entry_price': avg_price if size != 0 else 0,
                        'unrealized_pnl': unrealized_pnl
                    }
            except (KeyError, IndexError, ValueError) as e:
                logger.error(f"Error parsing Bybit position response: {e}")
        return {'size': 0, 'side': 'None', 'entry_price': 0, 'unrealized_pnl': 0}
    
    def place_order(self, symbol, side, qty, order_type="Market"):
        """Разместить ордер"""
        # Для Bybit используем более точное округление
        qty = float(Decimal(str(qty)).quantize(Decimal('0.1'), rounding=ROUND_DOWN))
        
        if qty < 0.1:
            logger.error(f"Quantity too small for Bybit: {qty} SOL")
            return None
        
        params = {
            'category': 'linear',
            'symbol': symbol,
            'side': side,  # "Buy" or "Sell"
            'orderType': order_type,
            'qty': str(qty)
        }
        
        logger.info(f"Placing Bybit order: {params}")
        result = self._make_request("POST", "/v5/order/create", params)
        
        if result and result.get('retCode') == 0:
            logger.info(f"Bybit order placed: {side} {qty} {symbol}")
            return result['result']['orderId']
        else:
            logger.error(f"Failed to place Bybit order: {result}")
            return None
    
    def cancel_order(self, symbol, order_id):
        """Отменить ордер"""
        params = {
            'category': 'linear',
            'symbol': symbol,
            'orderId': order_id
        }
        
        result = self._make_request("POST", "/v5/order/cancel", params)
        if result and result.get('retCode') == 0:
            logger.info(f"Bybit order cancelled: {order_id}")
            return True
        else:
            logger.error(f"Failed to cancel Bybit order: {result}")
            return False

class BinanceFuturesAPI:
    def __init__(self, api_key, secret_key):
        self.api_key = api_key
        self.secret_key = secret_key
        self.base_url = BINANCE_BASE_URL
        
    def _generate_signature(self, params):
        query_string = urlencode(params)
        return hmac.new(
            self.secret_key.encode('utf-8'),
            query_string.encode('utf-8'),
            hashlib.sha256
        ).hexdigest()
    
    def _make_request(self, method, endpoint, params=None, signed=False):
        if params is None:
            params = {}
            
        if signed:
            params['timestamp'] = int(time.time() * 1000)
            params['signature'] = self._generate_signature(params)
            
        headers = {'X-MBX-APIKEY': self.api_key}
        url = f"{self.base_url}{endpoint}"
        
        try:
            if method == "GET":
                response = requests.get(url, params=params, headers=headers, timeout=10)
            elif method == "POST":
                response = requests.post(url, data=params, headers=headers, timeout=10)
            elif method == "DELETE":
                response = requests.delete(url, params=params, headers=headers, timeout=10)
                
            response.raise_for_status()
            return response.json()
        except Exception as e:
            logger.error(f"Binance API error: {e}")
            return None
    
    def get_ticker_price(self, symbol):
        """Получить текущую цену"""
        try:
            response = requests.get(f"{self.base_url}/fapi/v1/ticker/price", 
                                  params={"symbol": symbol}, timeout=10)
            data = response.json()
            return float(data['price'])
        except Exception as e:
            logger.error(f"Error getting Binance price: {e}")
        return None
    
    def set_leverage(self, symbol, leverage):
        """Установить плечо"""
        params = {
            'symbol': symbol,
            'leverage': leverage
        }
        result = self._make_request("POST", "/fapi/v1/leverage", params, signed=True)
        if result:
            logger.info(f"Binance leverage set to {leverage}x for {symbol}")
            return True
        return False
    
    def get_balance(self):
        """Получить баланс USDT"""
        result = self._make_request("GET", "/fapi/v2/balance", signed=True)
        if result:
            for balance in result:
                if balance['asset'] == 'USDT':
                    return float(balance['availableBalance'])
        return 0
    
    def get_position(self, symbol):
        """Получить информацию о позиции"""
        result = self._make_request("GET", "/fapi/v2/positionRisk", signed=True)
        if result:
            for position in result:
                if position['symbol'] == symbol:
                    return {
                        'size': float(position['positionAmt']),
                        'entry_price': float(position['entryPrice']) if float(position['positionAmt']) != 0 else 0,
                        'unrealized_pnl': float(position['unRealizedProfit'])
                    }
        return {'size': 0, 'entry_price': 0, 'unrealized_pnl': 0}
    
    def place_order(self, symbol, side, quantity, order_type="MARKET"):
        """Разместить ордер"""
        # Для Binance округляем до 0.1 чтобы соответствовать Bybit
        quantity = float(Decimal(str(quantity)).quantize(Decimal('0.1'), rounding=ROUND_DOWN))
        
        if quantity < 0.1:
            logger.error(f"Quantity too small for Binance: {quantity} SOL")
            return None
        
        params = {
            'symbol': symbol,
            'side': side,
            'type': order_type,
            'quantity': quantity
        }
        
        logger.info(f"Placing Binance order: {params}")
        result = self._make_request("POST", "/fapi/v1/order", params, signed=True)
        
        if result and 'orderId' in result:
            logger.info(f"Binance order placed: {side} {quantity} {symbol}")
            return result['orderId']
        else:
            logger.error(f"Failed to place Binance order: {result}")
            return None
    
    def cancel_order(self, symbol, order_id):
        """Отменить ордер"""
        params = {
            'symbol': symbol,
            'orderId': order_id
        }
        
        result = self._make_request("DELETE", "/fapi/v1/order", params, signed=True)
        if result:
            logger.info(f"Binance order cancelled: {order_id}")
            return True
        else:
            logger.error(f"Failed to cancel Binance order: {result}")
            return False

# ==================== ОСНОВНАЯ СТРАТЕГИЯ ====================

class ArbitrageStrategy:
    def __init__(self):
        self.bybit = BybitPerpetualAPI(BYBIT_API_KEY, BYBIT_SECRET_KEY)
        self.binance = BinanceFuturesAPI(BINANCE_API_KEY, BINANCE_SECRET_KEY)
        
        self.in_position = False
        self.bybit_order_id = None
        self.binance_order_id = None
        self.entry_spread = 0
        self.sol_quantity = 0
        self.position_type = None
        
    def calculate_spread(self, bybit_price, binance_price):
        """Вычислить спред в процентах (Binance - Bybit) / Bybit * 100"""
        if bybit_price and binance_price:
            spread_percent = ((binance_price - bybit_price) / bybit_price) * 100
            return spread_percent
        return 0
    
    def calculate_position_size(self, price):
        """Вычислить размер позиции в SOL для торговли на 500 USD"""
        raw_quantity = POSITION_SIZE_USD / price
        # Округляем до 0.1 для Bybit совместимости
        return float(Decimal(str(raw_quantity)).quantize(Decimal('0.1'), rounding=ROUND_DOWN))
    
    def check_arbitrage_opportunity(self):
        """Проверить возможность арбитража"""
        bybit_price = self.bybit.get_ticker_price(BYBIT_SYMBOL)
        binance_price = self.binance.get_ticker_price(BINANCE_SYMBOL)
        
        if not bybit_price or not binance_price:
            logger.warning("Unable to get prices from exchanges")
            return False, 0, 0, 0
        
        spread_percent = self.calculate_spread(bybit_price, binance_price)
        
        # Логируем текущий спред
        logger.info(f"Current spread: {spread_percent:.3f}% | Bybit: ${bybit_price:.4f} | Binance: ${binance_price:.4f}")
        
        # Проверяем условия входа в позицию
        if not self.in_position and abs(spread_percent) >= MIN_SPREAD_PERCENT:
            return True, spread_percent, bybit_price, binance_price
        
        # Проверяем условия выхода из позиции
        if self.in_position and abs(spread_percent) <= 0.1:
            return True, spread_percent, bybit_price, binance_price
            
        return False, spread_percent, bybit_price, binance_price
    
    def enter_arbitrage_position(self, spread_percent, bybit_price, binance_price):
        """Войти в арбитражную позицию"""
        try:
            self.sol_quantity = self.calculate_position_size(bybit_price)
            logger.info(f"Calculated SOL quantity: {self.sol_quantity}")
            
            if spread_percent > MIN_SPREAD_PERCENT:  # Binance дороже Bybit
                logger.info(f"Entering LONG Bybit / SHORT Binance position. Spread: {spread_percent:.3f}%")
                
                # Лонг на Bybit (покупаем)
                bybit_order = self.bybit.place_order(BYBIT_SYMBOL, "Buy", self.sol_quantity)
                
                if bybit_order:
                    # Шорт на Binance (продаем)
                    binance_order = self.binance.place_order(BINANCE_SYMBOL, "SELL", self.sol_quantity)
                    
                    if binance_order:
                        # Оба ордера успешны
                        self.in_position = True
                        self.bybit_order_id = bybit_order
                        self.binance_order_id = binance_order
                        self.entry_spread = spread_percent
                        self.position_type = "LONG_BYBIT_SHORT_BINANCE"
                        logger.info(f"Successfully entered arbitrage position. Entry spread: {spread_percent:.3f}%")
                        return True
                    else:
                        # Binance не удался, отменяем Bybit
                        logger.error("Binance order failed, cancelling Bybit order")
                        self.bybit.cancel_order(BYBIT_SYMBOL, bybit_order)
                        return False
                else:
                    logger.error("Bybit order failed")
                    return False
            
            elif spread_percent < -MIN_SPREAD_PERCENT:  # Bybit дороже Binance
                logger.info(f"Entering SHORT Bybit / LONG Binance position. Spread: {spread_percent:.3f}%")
                
                # Шорт на Bybit (продаем)
                bybit_order = self.bybit.place_order(BYBIT_SYMBOL, "Sell", self.sol_quantity)
                
                if bybit_order:
                    # Лонг на Binance (покупаем)
                    binance_order = self.binance.place_order(BINANCE_SYMBOL, "BUY", self.sol_quantity)
                    
                    if binance_order:
                        # Оба ордера успешны
                        self.in_position = True
                        self.bybit_order_id = bybit_order
                        self.binance_order_id = binance_order
                        self.entry_spread = spread_percent
                        self.position_type = "SHORT_BYBIT_LONG_BINANCE"
                        logger.info(f"Successfully entered arbitrage position. Entry spread: {spread_percent:.3f}%")
                        return True
                    else:
                        # Binance не удался, отменяем Bybit
                        logger.error("Binance order failed, cancelling Bybit order")
                        self.bybit.cancel_order(BYBIT_SYMBOL, bybit_order)
                        return False
                else:
                    logger.error("Bybit order failed")
                    return False
                    
        except Exception as e:
            logger.error(f"Error entering position: {e}")
            
        return False
    
    def exit_arbitrage_position(self, spread_percent, bybit_price, binance_price):
        """Выйти из арбитражной позиции"""
        try:
            logger.info(f"Exiting arbitrage position. Current spread: {spread_percent:.3f}%, Entry spread: {self.entry_spread:.3f}%")
            
            if self.position_type == "LONG_BYBIT_SHORT_BINANCE":
                # Закрываем лонг на Bybit (продаем)
                bybit_exit_order = self.bybit.place_order(BYBIT_SYMBOL, "Sell", self.sol_quantity)
                # Закрываем шорт на Binance (покупаем)
                binance_exit_order = self.binance.place_order(BINANCE_SYMBOL, "BUY", self.sol_quantity)
                
            elif self.position_type == "SHORT_BYBIT_LONG_BINANCE":
                # Закрываем шорт на Bybit (покупаем)
                bybit_exit_order = self.bybit.place_order(BYBIT_SYMBOL, "Buy", self.sol_quantity)
                # Закрываем лонг на Binance (продаем)
                binance_exit_order = self.binance.place_order(BINANCE_SYMBOL, "SELL", self.sol_quantity)
            
            if bybit_exit_order and binance_exit_order:
                profit_percent = abs(self.entry_spread) - abs(spread_percent)
                logger.info(f"Successfully exited position. Estimated profit: {profit_percent:.3f}%")
                
                # Сбрасываем состояние
                self.in_position = False
                self.bybit_order_id = None
                self.binance_order_id = None
                self.entry_spread = 0
                self.sol_quantity = 0
                self.position_type = None
                return True
                
        except Exception as e:
            logger.error(f"Error exiting position: {e}")
            
        return False
    
    def print_status(self):
        """Вывести текущий статус"""
        bybit_balance = self.bybit.get_balance()
        binance_balance = self.binance.get_balance()
        bybit_position = self.bybit.get_position(BYBIT_SYMBOL)
        binance_position = self.binance.get_position(BINANCE_SYMBOL)
        
        logger.info(f"=== STATUS ===")
        logger.info(f"In Position: {self.in_position}")
        logger.info(f"Bybit Balance: {bybit_balance:.2f} USDT")
        logger.info(f"Binance Balance: {binance_balance:.2f} USDT")
        logger.info(f"Bybit Position: {bybit_position['size']:.3f} SOL ({bybit_position['side']})")
        logger.info(f"Binance Position: {binance_position['size']:.3f} SOL")
        if self.in_position:
            logger.info(f"Entry Spread: {self.entry_spread:.3f}%")
            logger.info(f"Position Type: {self.position_type}")
            logger.info(f"SOL Quantity: {self.sol_quantity:.3f}")
        logger.info(f"==============")
    
    def run(self):
        """Запустить стратегию"""
        logger.info("Starting SOL Perpetual/Futures Arbitrage Strategy")
        logger.info(f"Min Spread: {MIN_SPREAD_PERCENT}%")
        logger.info(f"Position size: {POSITION_SIZE_USD} USDT on each exchange")
        
        self.print_status()
        
        while True:
            try:
                opportunity, spread_percent, bybit_price, binance_price = self.check_arbitrage_opportunity()
                
                if opportunity:
                    if not self.in_position and abs(spread_percent) >= MIN_SPREAD_PERCENT:
                        # Входим в позицию
                        self.enter_arbitrage_position(spread_percent, bybit_price, binance_price)
                        
                    elif self.in_position and abs(spread_percent) <= 0.1:
                        # Выходим из позиции
                        self.exit_arbitrage_position(spread_percent, bybit_price, binance_price)
                
                # Выводим статус каждые 60 секунд
                if int(time.time()) % 60 == 0:
                    self.print_status()
                
                time.sleep(CHECK_INTERVAL)
                
            except KeyboardInterrupt:
                logger.info("Strategy stopped by user")
                break
            except Exception as e:
                logger.error(f"Unexpected error: {e}")
                time.sleep(CHECK_INTERVAL)

# ==================== ЗАПУСК ====================

if __name__ == "__main__":
    strategy = ArbitrageStrategy()
    strategy.run()

2025-07-03 18:24:02,070 - INFO - Starting SOL Perpetual/Futures Arbitrage Strategy
2025-07-03 18:24:02,074 - INFO - Min Spread: 0.02%
2025-07-03 18:24:02,075 - INFO - Position size: 500 USDT on each exchange
2025-07-03 18:24:02,498 - ERROR - Bybit API error: 401 Client Error: API key is invalid. for url: https://api.bybit.com/v5/account/wallet-balance?accountType=CONTRACT
2025-07-03 18:24:02,800 - ERROR - Binance API error: 401 Client Error: Unauthorized for url: https://fapi.binance.com/fapi/v2/balance?timestamp=1751577842500&signature=88c95a7cb5543ff4ca6ed359aa70a05c9bd85df72c756ca86d40b4613b3dc35a
2025-07-03 18:24:03,210 - ERROR - Bybit API error: 401 Client Error: API key is invalid. for url: https://api.bybit.com/v5/position/list?category=linear&symbol=SOLUSDT
2025-07-03 18:24:03,533 - ERROR - Binance API error: 401 Client Error: Unauthorized for url: https://fapi.binance.com/fapi/v2/positionRisk?timestamp=1751577843210&signature=aa3b25882761af2374f1116d4d063406e47dd4a87f5de3a5063